In [1]:
import torch
import torchvision
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### data loader

In [2]:
transforms = []
transforms += [torchvision.transforms.Resize(224)]
transforms += [torchvision.transforms.RandomCrop(200)]
transforms += [torchvision.transforms.RandomHorizontalFlip()]
transforms += [torchvision.transforms.RandomVerticalFlip()]
transforms += [torchvision.transforms.RandomRotation(degrees=25)]
transforms += [torchvision.transforms.ToTensor()]
transforms += [torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]
transforms = torchvision.transforms.Compose(transforms)
trainset = torchvision.datasets.ImageFolder(root='../../data/plant-seed/data/train/', transform=transforms)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True, num_workers=4)

In [3]:
transforms = []
transforms += [torchvision.transforms.Resize(200)]
transforms += [torchvision.transforms.ToTensor()]
transforms += [torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]
transforms = torchvision.transforms.Compose(transforms)
validset = torchvision.datasets.ImageFolder(root='../../data/plant-seed/data/valid/', transform=transforms)
validloader = torch.utils.data.DataLoader(validset, batch_size=16, shuffle=False, num_workers=4)

### resnet-34

In [4]:
model = torchvision.models.resnet50(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

model.fc = torch.nn.Linear(2048, 12)

criterion = torch.nn.CrossEntropyLoss()
optimizer= torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

### optimal learning rate

In [ ]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True, num_workers=4)
learning_rates = [10**x/100000000 for x in range(0,10)]

In [ ]:
losses = []

for learning_rate in learning_rates:
    # assign learning rate
    for param_group in optimizer.param_groups:
        param_group['lr'] = learning_rate
    # read data
    for i in range(6):
        _loss = 0.0 
        inputs, labels = next(iter(trainloader))
        if torch.cuda.is_available():
            inputs = Variable(inputs.cuda())
            labels = Variable(labels.cuda())
        else:
            inputs = Variable(inputs)
            labels = Variable(labels)

        # calculate loss and output
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        # update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _loss += loss.data[0]
    losses.append(_loss)
    
plt.figure(figsize=(10,5))
plt.plot(losses)
plt.xticks(range(len(losses)), learning_rates[:3])
plt.title('LR vs Loss')
plt.show()

### tune model

In [ ]:
learning_rates = [10**x/100000000 for x in range(0,10)]
criterion = torch.nn.CrossEntropyLoss()
optimizer= torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

In [16]:
def score(model, dataloader):
    calc_loss = 0.
    calc_correct = 0.
    calc_count = 0.
    for data in dataloader: 
        print('hi')
        inputs, labels = data
        if torch.cuda.is_available():
            inputs = Variable(inputs.cuda())
            labels = Variable(labels.cuda())
        else:
            inputs = Variable(inputs)
            labels = Variable(labels)
        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)
        calc_loss += loss.data[0]
        calc_correct += torch.sum(preds == labels.data)
        calc_count += outputs.data.shape[0]
    return calc_loss, calc_correct / calc_count, calc_count

In [17]:
score(model, trainloader)

hi
hi
hi
hi
hi
hi
hi


OSError: Traceback (most recent call last):
  File "/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/PIL/ImageFile.py", line 212, in load
    s = read(self.decodermaxblock)
  File "/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/PIL/PngImagePlugin.py", line 592, in load_read
    cid, pos, length = self.png.read()
  File "/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/PIL/PngImagePlugin.py", line 113, in read
    length = i32(s)
  File "/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/PIL/_binary.py", line 76, in i32be
    return unpack(">I", c[o:o+4])[0]
struct.error: unpack requires a buffer of 4 bytes

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 40, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/

In [ ]:
def train(trainloader, validloader, epochs, interval):
    for epoch in range(10):
        # read data
        loss = 10
        for data in trainloader: 
            inputs, labels = data
            if torch.cuda.is_available():
                inputs = Variable(inputs.cuda())
                labels = Variable(labels.cuda())
            else:
                inputs = Variable(inputs)
                labels = Variable(labels)
            # calculate loss and output
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            # update weights
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        if epoch % interval == 0:
            continue
            
        return model